In [35]:
from pyspark import SparkContext,SparkConf
sc = SparkContext.getOrCreate()

In [36]:
spark

In [37]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.pipeline import Pipeline, PipelineModel
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [38]:
ls

 Volume in drive C is Acer
 Volume Serial Number is 3498-618E

 Directory of C:\Users\verma\PycharmProjects\pyspark

19-01-2020  16:47    <DIR>          .
19-01-2020  16:47    <DIR>          ..
19-01-2020  11:06    <DIR>          .idea
19-01-2020  16:22    <DIR>          .ipynb_checkpoints
19-01-2020  16:25         1,215,506 CleanCreditScoring.csv
20-09-2019  00:04       150,828,752 creditcard-fraud.csv
22-11-2019  04:11    <DIR>          Movies25M
19-01-2020  15:50           697,798 Pride_and_Prejudice.txt
19-01-2020  11:22               362 pyspark101.py
19-01-2020  12:01            37,134 Spark101.ipynb
19-01-2020  15:09            13,599 Spark102.ipynb
19-01-2020  15:45             8,277 Spark103 _Stockdataset.ipynb
19-01-2020  16:21            36,836 Spark104_Mapreduce.ipynb
19-01-2020  16:47             8,728 Spark105.ipynb
03-08-2018  00:58        25,851,020 stocks.csv
04-07-2016  13:59           179,591 train.txt
19-01-2020  11:20           108,032 winutils.exe
              12

In [39]:
df = (spark.read
      .option("inferSchema", True)
      .option("header", True)
      .csv("creditcard-fraud.csv"))

In [45]:
#df.limit(10).toPandas()

In [33]:
feature_columns = [col for col in df.columns if col.startswith("V")]
print(feature_columns)

['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28']


In [46]:
vectorizer = VectorAssembler(inputCols = feature_columns, outputCol="features")
vectorizer.transform(df).select("features", "Class")

DataFrame[features: vector, Class: int]

In [47]:
est = RandomForestClassifier()
est.setMaxDepth(5)
est.setLabelCol("Class")

RandomForestClassifier_256e94b00b86

In [49]:
print(est.explainParams())

cacheNodeIds: If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users can set how often should the cache be checkpointed or disable it by setting checkpointInterval. (default: False)
checkpointInterval: set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext. (default: 10)
featureSubsetStrategy: The number of features to consider for splits at each tree node. Supported options: 'auto' (choose automatically for task: If numTrees == 1, set to 'all'. If numTrees > 1 (forest), set to 'sqrt' for classification and to 'onethird' for regression), 'all' (use all features), 'onethird' (use 1/3 of the features), 'sqrt' (use sqrt(number of features)), 'log2' (use log2(number of features)), 

In [50]:
df_train, df_test = df.randomSplit(weights=[0.7, 0.3], seed = 1)

In [ ]:
pipeline = Pipeline()
pipeline.setStages([vectorizer, est])
model = pipeline.fit(df_train)

In [ ]:
df_test_pred = model.transform(df_test)